In [ ]:
!pip install transformers datasets accelerate nvidia-ml-py3 --quiet
# !git clone https://github.com/huggingface/transformers --quiet
# %cd transformers
# !pip install . --quiet
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

In [ ]:
from datasets import Dataset, DatasetDict
import regex as re
import torch
from tqdm.notebook import tqdm
import pandas as pd
from pynvml import *
from transformers import (AutoTokenizer,
                          DataCollatorWithPadding, 
                          AutoModelForSequenceClassification, 
                          TrainingArguments, 
                          Trainer)

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

path = r'/content/drive/MyDrive/pdf_parsers/parser_classifier/afpdf_data_v0.csv'

df = pd.read_csv(path)
df = df.sample(frac=1).reset_index()
df = df[['header', 'text']]
df = df.rename(columns={'header': 'label'})

dataset = Dataset.from_pandas(df)

train_test_valid = dataset.train_test_split(test_size=0.4)
# Split the test + valid in half test, half valid
test_valid = train_test_valid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
ds = DatasetDict({
    'train': train_test_valid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']})
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 91929
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 30644
    })
    valid: Dataset({
        features: ['label', 'text'],
        num_rows: 30643
    })
})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2).to("cuda")

tokenized_ds = ds.map(preprocess_function, batched=True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classi

  0%|          | 0/92 [00:00<?, ?ba/s]

  0%|          | 0/31 [00:00<?, ?ba/s]

  0%|          | 0/31 [00:00<?, ?ba/s]

In [ ]:
model_name = f"distilbert-base-uncased-finetuned-header-classifier"

training_args = TrainingArguments(
                                  output_dir=model_name,
                                  push_to_hub=True,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=4,
                                  per_device_eval_batch_size=4,
                                  num_train_epochs=5,
                                  weight_decay=0.01,
                                  gradient_accumulation_steps=16,
                                  evaluation_strategy='epoch',
                                  # gradient_checkpointing=True,
                                  fp16=True,
                                  optim="adafactor"
                                  )

trainer = Trainer(
                  model=model,
                  args=training_args,
                  train_dataset=tokenized_ds["train"],
                  eval_dataset=tokenized_ds["test"],
                  tokenizer=tokenizer,
                  data_collator=data_collator
                  )

trainer.train()
# trainer.push_to_hub()
trainer.evaluate()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("alk/distilbert-base-uncased-finetuned-header-classifier")
model = AutoModelForSequenceClassification.from_pretrained("alk/distilbert-base-uncased-finetuned-header-classifier").to("cuda")

loading file https://huggingface.co/alk/distilbert-base-uncased-finetuned-header-classifier/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/32cb779d128bd984dd60656858489a4ab0586ce6f1916529d20e4b1652145e86.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/alk/distilbert-base-uncased-finetuned-header-classifier/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/419edd542552d910682a3a8fc3918133222c34f606fdd6f43a46853f5c908fe6.848c414913cfee271695b8761d3e947fb18a724fbad549de63228b20e5f2d615
loading file https://huggingface.co/alk/distilbert-base-uncased-finetuned-header-classifier/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/alk/distilbert-base-uncased-finetuned-header-classifier/resolve/main/special_tokens_map.json from cache at /root/.cache/huggingface/transformers/e50645bff0d24e8e1187f092e83d700c12cf15c93b7dcf0d25698c7c0d3f4a97.7da70648c6cb9

In [ ]:
preds = []
text_items = []
actual_items = []
raw_preds = []
for i in tqdm(range(0, len(ds['test']))):
    text = ds['test'][i]['text']
    text_items.append(text)
    actual = ds['test'][i]['label']
    actual_items.append(actual)
    inputs = tokenizer(ds['test'][i]['text'], padding = True, truncation = True, max_length = 512, return_tensors = 'pt').to("cuda")
    with torch.no_grad():
        logits = model(**inputs).logits
        predicted_class_id = logits.argmax().item()
        pred = model.config.id2label[predicted_class_id]
        clean_pred = re.sub('LABEL_', '', str(pred))
        preds.append(int(clean_pred))
        raw_preds.append(logits)

d = {
    'raw_preds': raw_preds,
    'text': text_items,
    'preds': preds,
    'actual': actual_items
     }

In [ ]:
df1 = pd.DataFrame(d)
# df2 = df1[0:200]
# df2.to_csv(r'HF_distilbert_classifier_results.csv', index=False)
df1

In [ ]:
df1['correct'] = df1['preds'] + df1['actual']

def check_accuracy(x):
    if x == 2 or x == 0: return 1
    if x == 1: return 0

df1['correct'] = df1['correct'].apply(lambda x: check_accuracy(x))
df1

In [ ]:
accuracy = round(sum(df1['correct'].to_list()) / len(df1['correct']), 3)
accuracy

0.999